In [118]:
import librosa
import librosa.display
import glob
from scipy.signal import butter, filtfilt, freqz
import numpy as np
import matplotlib.pyplot as plt
import json
import os
import re
import platform
import subprocess as sp

In [2]:
def butter_lowpass(cutoff, fs, order=5):
    nyq = 0.5 * fs
    normal_cutoff = cutoff / nyq
    b, a = butter(order, normal_cutoff, btype='low', analog=False)
    return b, a

def butter_lowpass_filter(data, cutoff, fs, order=5):
    b, a = butter_lowpass(cutoff, fs, order)
    y = filtfilt(b, a, data)
    return np.array(y)

In [3]:
def plot_audio(data):
    plt.figure(figsize=(10, 4))
    # Variables for plotting
    n = len(data)           # total number of samples
    T = n / fs              # seconds
    t = np.linspace(0, T, n)
    plt.plot(t, data, linewidth=.2)
    plt.show()

In [144]:
def get_MFCC_per_folder(path, cutoff=400, order=6, top_db=20):
    wav_file_list = [f.path for f in os.scandir(path)]
    mfcc_dataset = {}
    for wav_file in wav_file_list:
        
        # Get identifier of the audio clip
        key = re.findall('\d+', wav_file)[0]
        sound, sampling_rate = librosa.load(wav_file)
        fs = sampling_rate   # sample rate, Hz
        
        # low-pass filter the sound by removing all frequencies
        # above the cutoff value (default=400 Hz)
        low_passed_sound = butter_lowpass_filter(sound, cutoff, fs, order)
        
        # Trim the silent parts of the speech signal
        low_passed_trimmed_sound, _ = librosa.effects.trim(low_passed_sound, top_db=top_db)
        
        # Calculate the MFCC values of the low-pass filtered and trimmed speech signal
        mfccs = librosa.feature.mfcc(y=low_passed_trimmed_sound, sr=sampling_rate, n_mfcc=12).tolist()
        mfcc_dataset[key] = mfccs
        
    return mfcc_dataset

In [128]:
def save_MFCC_as_json(MFCC_data, filepath):
    with open(filepath, 'w', encoding='utf-8') as f:
        json.dump(MFCC_data, f, ensure_ascii=False, indent=4)
    return None

def set_path_seperator():
    path_sep = "/"
    if platform.system() == "Windows":
        path_sep = "\\"
    return path_sep


def get_paths_of_language(source_path, language_folder, sub_folders):
    languages = [f.name for f in os.scandir(source_path)]
    if not language_folder in languages:
        print('You do not have any data of this language!')
        return None
    else:
        paths = []
        for folder in sub_folders:
            if platform.system() == 'Windows':
                paths.append(source_path + "\\" + language_folder + "\\" + folder + "\\" + '\\')
            else:
                paths.append(source_path + "/" + language_folder + "/" + folder + "/")
        return paths
    
def convert_to_windows_path(path):
    return "\\".join(path.split('/'))

In [158]:
# IMPORTANT: in case you have downloaded the data in a different audio format (for example in .mp3)
# convert all audio files that you want to use into .wav files

paths = get_paths_of_language('Downsampled', 'dummy_clips',\
                              ['train_wav_clips', 'test_wav_clips', 'validate_wav_clips'])

# Variables needed for the filter
order = 6
cutoff = 400            # desired cutoff frequency of the filter, Hz

MFCC_train_set = get_MFCC_per_folder(paths[0])
MFCC_test_set = get_MFCC_per_folder(paths[1])
MFCC_validate_set = get_MFCC_per_folder(paths[2])

In [159]:
path_sep = set_path_seperator()
language_folder = paths[0].split(path_sep)[1]
dst_folder = "Downsampled" + path_sep + language_folder + path_sep + 'MFCC_json_files' + path_sep

sp.run("rm -r " + dst_folder, shell=True)
sp.run("mkdir " + dst_folder, shell=True)

# save data in json files
save_MFCC_as_json(MFCC_train_set, dst_folder + 'MFCC_train.json')
save_MFCC_as_json(MFCC_test_set, dst_folder + 'MFCC_test.json')
save_MFCC_as_json(MFCC_validate_set, dst_folder + 'MFCC_validate.json')